# 5. MEMORY
랭체인에는 5가지 정도 종류의 메모리가 있는데 각자 저장 방식도 다르고 각자만의 장단점이 있다.
`챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없다``.
유저가 자신의 이름을 말하거나 이전 질무에 이어지는 질문을 해도, 챗봇은 메모리, 즉 기억력이 없기 때문에
대화를 이해할 수 있는 능력이 없다.
오픈API에서 제공하는 기본 API는 랭체인 없이 사용할 수 있는데, 메모리를 지원하지 않는다.
한마디로 stateless.
즉, `모델에게 어떤 말을 건네면 모델은 답을 한 직후에 모든 대화 내용을 잊어버리게 된다.`
아무런 내용도 저장하지 않는다는 말이다.
챗GPT에는 메모리가 탑재되어 있기 때문에 실제로 어떤 사람과 얘기하고 있다는 느낌을 들게한다.
챗봇이 이전의 대화 내용이나 질문을 기억하고 답할 수 있으므로...

### 강의 내용
일단 각 메모리의 종류를 알아보고 차이점을 살펴본 후,
랭체인에 메모리를 탑재시키는 방법을 배워보자.

## 5.0 ConversationBufferMemory
첫번째 메모리는 `Conversation Buffer Memory`라고하는데, 이메모리는 되게 단순하다.
그냥 단순히 이전 대화 내용 전체를 저장하는 메모리이다.
### 단점
대화 내용이 길어질수록 `메모리도 계속 커지므로 비효율적`이다.
위에서 설명했듯이 `모델 자체에는 메모리`가 없다. 
그래서 우리가 모델에게 요청을 보낼 때 `이전 대화 기록 전체를 같이 보내야 된다`.
그래야지만 모델이 전에 일어났던 대화를 보고 이해를 할 수 있다.
유저와 AI의 대화가 길어질수록 우리가 모델에게 `매번 보내야 될 대화 기록이 길어진다`는 뜻이다.
상당히 `비효율적`이고 당연히 `비용도 많이 발생`한다.

비효율적이긴 하지만, 그래도 제일 이해하기 쉬운 메모리니까 한번 살펴보자.


In [7]:
from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory()
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})




{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [8]:
memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})
# 아래와 같에 메모리에 모든 이력이 쌓이므로 대화가 길어질수록 비효율적이다.

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

## 5.1 ConversationBufferWindowMemory
5.0 에서는 ConversationBufferMemory에 대해서 배웠다.
ConversationBufferMemory와 다르게 ConversationBufferWindowMemory는 `대화의 특정 부분만을 저장하는 메모리`이다.
예를 들어 가장 `최근 5개의 메시지만 저장`한다고 하면 6번째 메시지가 추가되었을 때,
`가장 오래된 메시지는 버려지는 방식`이다.

ConversationBufferWindowMemory는 `대화의 특정 부분, 저장범위는 우리가 직접 설정` 할 수 있다.
### 장점
메모리를 특정 크기로 유지할 수 있다는게 이메모리의 큰 장점이다.
이 메모리의 장점은 모든 대화 내용을 저장하지 않아도 된다는 점
### 단점
챗봇이 `전체 대화`가 아닌 `최근 대화에만 집중`한다는 것이다.


In [9]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1, 1)


In [10]:
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)


In [11]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [12]:
add_message(5, 5)
memory.load_memory_variables({})


{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

### 실행 결과
위 결과를 보면 마지막에 5번쨰 메모리를 추가하면 `첫번째 메모리 데이터가 제거`된 것을 알수 있다.
`메모리의 사이즈가 계속 늘어나지 않는다는게 장점`이지만,
`단점은 챗봇이 예전 대화를 기억하기 힘들다는 점`이다.
`최근에 일어난 대화에만 집중`한다는 것이다.


## 5.2 ConversationSummaryMemory
`ConversationSummaryMemory`는 `llm`을 사용한다.
ConversationSummaryMemory는 message 그대로 저장하는 것이 아니라
conversation의 요약을 자체적으로 해주는 것이다.
이 부분은 유용하다. 특히 매우 긴 Conversation이 있는 경우에.
초반에는 ConversationSummaryMemory는 이전보다 더 많은 토큰과 저장공간을 차지하게 된다.
왜냐하면 위에서는 'Hi', 'How are you?'와 같은 짧은 메시지만을 저장했지만,
만약 ConversationSummaryMemory가 사람과 대화를 하고, 사람이 자신을 소개하면서 AI가 답하게 되는 경우를 생각해보자.
그러면 더 오랜 시간이 걸릴 것이다. 초반에는 더 오래 걸릴것이다.
그러나 conversation은 버퍼 메모리를 사용하고 있어서 대화가 진행될수록 저장된 모든 메시지가 매우 많아지면서 연결될 것이다.
`conversation의 메시지가 많아질수록 ConversationSummaryMessage의 도움을 받아서 요약`하는 것이 `토큰의 양도 줄어들면서 훨씬 나은 방법`이 될것이다.

Conversation이 많아질수록 이전처럼 모든 메세지를 버퍼에 저장하는 것 보다 Conversation을 사용하여 요약하는 것이 훨씬 낫다는 걸 알게 될 것이다.




In [13]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 메모리를 llm을 이요한다. 메모리를 실행하는데 비용이 든다는 뜻이다.
memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Jungwon, I live in South Korea", "Wow that is so cool!")




In [14]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [15]:
get_history()

{'history': 'The human introduces themselves as Jungwon from South Korea. The AI finds this information cool and expresses a desire to visit South Korea because it is pretty.'}

## 5.3 ConversationSummaryBufferMemory
`ConversationSummaryMemory`, `ConversationBuffer`의 결합이다.
이것이 하는 일은, `메모리에 보내온 메시지의 수를 저장`하는 것이다.
또한, 우리가 limit에 다다른 순간에, 그냥 무슨 일이 일어났는지 잊어버리는 것 대신에
오래된 메시지들을 summarize(요약)할 것이다.
* 이것이 의미하는 바는 `가장 최근의 상호작용을 계속 추적`한다는 것이다.
    * 가장 최근 및 가장 오래전에 주고 받은 메시지가 모두 `잊혀지지 않고 요약이 되는 것`이다.


In [24]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150, #가능한 메시지 토큰 수의 최대값.
    return_messages=True # 이것을 채팅 모델에 사용할 것이기 때문에 True로 하는 것이 좋다.
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Jungwon, I live in South Korea", "Wow that is so cool!")


In [25]:
get_history()

{'history': [HumanMessage(content="Hi I'm Jungwon, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [26]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [27]:
get_history()

{'history': [HumanMessage(content="Hi I'm Jungwon, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!')]}

In [28]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [29]:
get_history()

{'history': [HumanMessage(content="Hi I'm Jungwon, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [34]:
add_message("How far is Brazil from Argentina?", "I don't know! Suer far!")

add_message("What is the capital of France?", "Paris!")

add_message("What is the capital of Germany?", "Berlin!")


In [35]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Jungwon from South Korea. The AI responds by expressing admiration for their location and wishing it could go there. When asked about the distance between Korea and Argentina, the AI admits it doesn\'t know but assumes it is "super far."'),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Suer far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Suer far!"),
  HumanMessage(content='What is the capital of France?'),
  AIMessage(content='Paris!'),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Suer far!"),
  HumanMessage(content='What is the capital of France?'),
  AIMessage(content='Paris!'),
  HumanMessage(content='What is the capital of Germany?'),
  AIMessage(content='Berlin!')]}

### 실행 결과
위 결과 처럼 메모리에 지정된 토큰수가 넘어가는 경우 `오래된 메시지들이 요약`된다.
`하지만, 이것은 우리가 지불하는 API를 사용하고 있다는 것.`

## 5.4 ConversationKGMemory (Conversation Knowledge Graph Memory)
이것도 LLM을 사용하는 memory class이다.
대화 중의 엔티티의 Knowledge graph를 만든다.
`가장 중요한 것들만 뽑아내는 요약본` 같은 것이다.

아래 코드는 Knowledge graph에서 히스토리를 가지고 오지 않고 엔티티를 가지고 오기 때문에 
get_history함수를 지웠다.

In [36]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True # 이것을 채팅 모델에 사용할 것이기 때문에 True로 하는 것이 좋다.
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

# def get_history():
#     return memory.load_memory_variables({})

add_message("Hi I'm Jungwon, I live in South Korea", "Wow that is so cool!")


In [37]:
memory.load_memory_variables({"input": "who is Jungwon"})

{'history': [SystemMessage(content='On Jungwon: Jungwon is a person. Jungwon lives in South Korea.')]}

In [38]:
add_message("Jungwon likes Kinchi", "Wow that is so cool!")

In [39]:
memory.load_memory_variables({"input": "What does Jungwon like"})

{'history': [SystemMessage(content='On Jungwon: Jungwon is a person. Jungwon lives in South Korea. Jungwon likes Kinchi.')]}

## ConversationTokenBufferMemory
Interaction의 최대 값을 가지고 있는 것 대신에 token의 총 양을 계산한다.
ConversationBufferWindow와 유사하다. 단지, Interaction 대신에 Token을 이용한다는 것.

## Entity
대화 중에 entity를 추출한다.
주목해야할 점은 여기에 정말 많은 데이터베이스를 기반으로 한 아주 많은 integration이 있다는 것이다.
Memory를 DB에 백업할 수 있다.

